# Day 4 part 1: Real-time with outbreak data

In [ ]:
knitr::opts_chunk$set(echo = TRUE)

# toggle to render full version 
render_snippet <- if (params$full_version) 'asis' else FALSE

## Introduction

This practical (in two parts) simulates the early assessment and
reconstruction of an Ebola Virus Disease (EVD) outbreak. It introduces various
aspects of analysis of the early stage of an outbreak, including case fatality
ratio (CFR), epicurves, growth rate
estimation, contact tracing data, delays, and estimates of transmissibility.

In [ ]:
# ### this is what we used to create the dataset:
# # assuming that cases are reported on the hospitalisation date
# data <- outbreaks::ebola_sim
# threshold_date <- as.Date("2014-07-01")
# linelist_20140701 <- data$linelist[data$linelist$date_of_hospitalisation <= threshold_date, ]
# # remove any outcome dates that are after present
# linelist_20140701$outcome[linelist_20140701$date_of_outcome > threshold_date] <- NA
# linelist_20140701$date_of_outcome[linelist_20140701$date_of_outcome > threshold_date] <- NA
# write.csv(linelist_20140701, "data/linelist_20140701.csv", row.names = FALSE) # then manually change to excel file
# ### NOTE FROM ZNK (2019-06-13)
# # Opening this file in excel to convert to an excel document caused two case
# # ids to be interpreted as numbers: 700e84 and 8940e5. It's actually a pretty
# # good example to use for highlighting the errors of excel, but a better way
# # to preserve these tags would be to use the rio package:
# # rio::export(data, 'path/to/file.xlsx')
# 
# contacts_20140701 <- data$contacts
# contacts_20140701 <- contacts_20140701[!(is.na(match(contacts_20140701$case_id, linelist_20140701$case_id)) | is.na(match(contacts_20140701$infector, linelist_20140701$case_id))), ]
# id <- intersect(contacts_20140701$infector, linelist_20140701$case_id)
# id_maybe_dead <- linelist_20140701$case_id[(linelist_20140701$case_id %in% id) &
#                             !(linelist_20140701$outcome %in% "Recover")]
# set.seed(1)
# change_source_to_funeral <- sample(which(contacts_20140701$infector %in% id_maybe_dead), 20) # add some funeral exposures as well
# contacts_20140701$source[change_source_to_funeral] <- "funeral"
# write.csv(contacts_20140701, "data/contacts_20140701.csv", row.names = FALSE) # then manually change to excel file


## Learning outcomes

By the end of this practical, you should be able to:

- Load and clean outbreak data in R ([part 1](./real-time-response-1.html))

- Estimate the case fatality ratio (CFR) ([part 1](./real-time-response-1.html))

- Compute and plot incidence from linelist ([part 1](./real-time-response-1.html))

- Estimate & interpret the growth rate & doubling time of the epidemic ([part 2](./real-time-response-2.html))

- Estimate the serial interval from data on pairs infector / infected individuals ([part 2](./real-time-response-2.html))

- Estimate & interpret the reproduction number of the epidemic ([part 2](./real-time-response-2.html))

- Forecast short-term future incidence ([part 2](./real-time-response-2.html))

- Reconstruct who infected whom using epidemiological and genetic data ([part 3](./real-time-response-3.html))

## A novel EVD outbreak in a fictional country in West Africa

A new EVD outbreak has been notified in a fictional country in West Africa. The Ministry of Health is in charge of coordinating the outbreak response, and have contracted you as a consultant in epidemic analysis to inform the response in real time.

## Required packages 

The following packages, available on CRAN or github, are needed for this analysis.
Install necessary packages as follows:

In [ ]:
# install.packages("remotes")
# install.packages("readxl")
# install.packages("outbreaks")
# install.packages("incidence")
# remotes::install_github("reconhub/epicontacts@ttree")
# install.packages("distcrete")
# install.packages("epitrix")
# remotes::install_github("annecori/EpiEstim")
# remotes::install_github("reconhub/projections")
# install.packages("ggplot2")
# install.packages("magrittr")
# install.packages("binom")
# install.packages("ape")
# install.packages("outbreaker2")
# install.packages("here")

Once the packages are installed, you may need to open a new R session. Then load the libraries as follows:

In [ ]:
library(readxl)
library(outbreaks)
library(incidence)
library(epicontacts)
library(distcrete)
library(epitrix)
library(EpiEstim)
library(projections)
library(ggplot2)
library(magrittr)
library(binom)
library(ape)
library(outbreaker2)
library(here)

## Early data (reading data into R)

You have been given the following linelist and contact data:

**linelist_20140701.xlsx**: a linelist containing case information up to the 1st July 2014; and

**contact_20140701.xlsx**: a list of contacts reported between cases up to the 1st July 2014. "infector" indicates a potential source of infection, and "case_id" the recipient of the contact.

To read into R, download these files and use the function `read_xlsx()` from the `readxl` package to import the data. Each import will create a data table stored as a `tibble` object. 

- Call the first one `linelist`, and  
- the second one `contacts`. 

For instance, you first command line could look like:  



In [ ]:
linelist <- read_excel(here("data","linelist_20140701.xlsx"), na = c("", "NA"))
contacts <- read_excel(here("data","contacts_20140701.xlsx"), na = c("", "NA"))

Take some time to look at the data and structure here. 

- Are the data and format similar to linelists that you have seen in the past? 
- If you were part of the outbreak investigation team, what other information might you want to collect?

In [ ]:
dim(linelist)
head(linelist)

- You may want to also collect data on date of report, age, household identifier, occupation, etc.

Note that for further analyses, you will need to make sure that all dates as
stored correctly as `Date` objects.  You can do this by using the function
`as.Date`, for example:  

In [ ]:
linelist$date_of_onset <- as.Date(linelist$date_of_onset, format = "%Y-%m-%d")

In [ ]:
linelist$date_of_infection <- as.Date(linelist$date_of_infection, format = "%Y-%m-%d")
linelist$date_of_hospitalisation <- as.Date(linelist$date_of_hospitalisation, format = "%Y-%m-%d")
linelist$date_of_outcome <- as.Date(linelist$date_of_outcome, format = "%Y-%m-%d")

The formatted data should then look like:

In [ ]:
head(linelist)
head(contacts)

## Data cleaning and descriptive analysis

Look more closely at the data contained in this `linelist`.   

- What do you notice?  

In [ ]:
head(linelist)
names(linelist)

You may notice that there are missing entries.  
An important step in analysis is to identify any mistakes in data entry. 
Although it can be difficult to assess errors in hospital names, we would expect the date of infection to always be before the date of symptom onset.  
  
Clean this dataset to remove any entries with negative or 0 day incubation periods.  

In [ ]:
## identify mistakes in data entry (negative incubation period)
mistakes <- which(linelist$date_of_onset <= linelist$date_of_infection)
mistakes
linelist[mistakes, ]


In [ ]:
## identify mistakes in data entry (negative incubation period)
mistakes <- which(linelist$date_of_onset <= linelist$date_of_infection)
mistakes
linelist[mistakes, ] # show just the first few entries where there is negative or 0 incubation times.

Save your "cleaned" linelist as a new object: `linelist_clean`  

In [ ]:
linelist_clean <- linelist[-mistakes, ]

What other negative dates or mistakes might you want to check if you had the full dataset?

In [ ]:

cat("- You may want to see if there are mistakes including but not restricted to: i) negative symptom onset to hospitalisation or outcome delays and ii) spelling errors such as hospital and names")  


## Calculating the case fatality ratio (CFR)

Here are the number of cases by outcome status. How would you calculate the CFR from this? 

In [ ]:
table(linelist_clean$outcome, useNA = "ifany")

Think about what to do with cases whose outcome is NA?

In [ ]:
n_dead <- sum(linelist_clean$outcome %in% "Death")
n_known_outcome <- sum(linelist_clean$outcome %in% c("Death", "Recover"))
n_all <- nrow(linelist_clean)
cfr <- n_dead / n_known_outcome
cfr_wrong <- n_dead / n_all

cfr_with_CI <- binom.confint(n_dead, n_known_outcome, method = "exact")
cfr_wrong_with_CI <- binom.confint(n_dead, n_all, method = "exact")

In [ ]:

cat("- Not properly accounting for the cases with unknown outcome status typically leads to underestimation of the CFR. This is particularly problematic early on in an outbreak where the final status of a large proportion of cases has not yet been observed.")  


## Looking at incidence curves

The first question we want to know is simply: how bad is it?. The first step of the analysis is descriptive - we want to draw an epidemic curve or epicurve. This visualises the incidence over time by date of symptom onset.

Using the package `incidence`, compute the daily incidence from the `linelist_clean` based on the dates of symptom onset. Store the result in an object called i_daily; the result should look like:  

In [ ]:
i_daily <- incidence(linelist_clean$date_of_onset) # daily incidence

In [ ]:
i_daily

plot(i_daily, border = "black")

You might notice that the incidence dates `i_daily$dates` stops on the last date where we have data on date of symptom onset (29th June 2014). However close inspection of the linelist shows that the last date in the linelist (of any entry) is in fact a bit later (1st July 2014). You can use the argument `last_date` in the `incidence` function to change this.  

In [ ]:

#extend last date:
i_daily <- incidence(linelist_clean$date_of_onset, 
                     last_date = as.Date(max(linelist_clean$date_of_hospitalisation, na.rm = TRUE)))
i_daily
plot(i_daily, border = "black")


Another issue is that it may be hard to interpret trends when looking at daily 
incidence, so also compute and plot the weekly incidence `i_weekly`, as follows:

In [ ]:
i_weekly <- incidence(linelist_clean$date_of_onset, interval = 7, 
                      last_date = as.Date(max(linelist_clean$date_of_hospitalisation, na.rm = TRUE)))
i_weekly

plot(i_weekly, border = "black")

## Save data and outputs

This is the end of [part 1](./real-time-response-1.html) of the practical. 
Before going on to [part 2](./real-time-response-2.html), you'll need to save the following objects:

In [ ]:
dir.create(here("data/clean")) # create clean data directory if it doesn't exist
saveRDS(i_daily, here("data/clean/i_daily.rds"))
saveRDS(i_weekly, here("data/clean/i_weekly.rds"))
saveRDS(linelist, here("data/clean/linelist.rds"))
saveRDS(linelist_clean, here("data/clean/linelist_clean.rds"))
saveRDS(contacts, here("data/clean/contacts.rds"))